In [2]:
!pip install scikeras[tensorflow]

  Using cached scikeras-0.12.0-py3-none-any.whl (27 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp39-cp39-win_amd64.whl (1.5 MB)
  Using cached tensorflow-2.12.1-cp39-cp39-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.12.1-cp39-cp39-win_amd64.whl (272.8 MB)
  Using cached protobuf-4.25.0-cp39-cp39-win_amd64.whl (413 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached absl_py-2.0.0-py3-none-any.whl (130 kB)
  Using cached libclang-16.0.6-py2.py3-none-win_amd64.whl (24.4 MB)
  Using cached tensorboard-2.12.3-py3-none-any.whl (5.6 MB)
  Using cached jax-0.4.20-py3-none-any.whl (1.7 MB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorflow_estimator-2.12.0-py2.py3-none-any.whl (440 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Us

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-storage 1.31.0 requires google-auth<2.0dev,>=1.11.0, but you have google-auth 2.23.4 which is incompatible.
google-cloud-core 1.7.1 requires google-auth<2.0dev,>=1.24.0, but you have google-auth 2.23.4 which is incompatible.
google-api-core 1.25.1 requires google-auth<2.0dev,>=1.21.1, but you have google-auth 2.23.4 which is incompatible.


In [4]:
!pip install mnist

In [6]:
!pip install keras

In [1]:
import numpy as np
import pandas as pd
import mnist
import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.utils import to_categorical
#from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
from scikeras.wrappers import KerasClassifier

In [3]:
#import images
train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

# Normalize the images.
train_images = (train_images / 255)
test_images = (test_images / 255)

# Flatten the images.
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

In [4]:
# Build the model.
def create_model(optimizer='adam'):
    model = Sequential([
      Dense(64, input_shape=(784,), activation='relu'),
      Dense(32, activation='relu'),
      Dense(10, activation='softmax')
    ])

    # Compile the model.
    model.compile(
      optimizer=optimizer,
      loss='categorical_crossentropy',
      metrics=['accuracy'],
    )
    return model

In [5]:
model = create_model()

# Train the model.
model.fit(train_images, to_categorical(train_labels), epochs=5, batch_size=32)

# Evaluate the model.
loss,metrics = model.evaluate(test_images, to_categorical(test_labels))
print('**************')
print(f'Loss - {np.round(loss,2)}, Metrics - {np.round(metrics,3)}')

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2933 - accuracy: 0.9148
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1348 - accuracy: 0.9600
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0967 - accuracy: 0.9717
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0770 - accuracy: 0.9764
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0912 - accuracy: 0.9747
**************
Loss - 0.09, Metrics - 0.975


In [6]:
# Predict on the first 10 test images.
predictions = model.predict(test_images[:10])

# Print our model's predictions.
print(f'Predictions: {np.argmax(predictions, axis=1)}')

# Check our predictions against the ground truths.
print(f'True labels: {test_labels[:10]}')

1/1 [==============================] - 0s 124ms/step
Predictions: [7 2 1 0 4 1 4 9 5 9]
True labels: [7 2 1 0 4 1 4 9 5 9]


### Picking params via GridSearchCV

In [7]:
batch_size = [ 500, 1000]
epochs = [5, 10]
optimizer = ['SGD', 'RMSprop', 'Adam']

In [8]:
model = KerasClassifier(create_model)
params= dict(batch_size=batch_size, epochs=epochs, optimizer=optimizer)
grid = GridSearchCV(model, params, refit=False, cv=3, scoring='accuracy')

grid_result = grid.fit(train_images, to_categorical(train_labels))
print(grid_result.best_score_, grid_result.best_params_)

Epoch 1/5
80/80 [==============================] - 2s 6ms/step - loss: 0.9567 - accuracy: 0.7258
Epoch 2/5
80/80 [==============================] - 0s 6ms/step - loss: 0.3292 - accuracy: 0.9079
Epoch 3/5
80/80 [==============================] - 1s 7ms/step - loss: 0.2602 - accuracy: 0.9262
Epoch 4/5
80/80 [==============================] - 0s 6ms/step - loss: 0.2227 - accuracy: 0.9371
Epoch 5/5
40/40 [==============================] - 0s 3ms/step
Epoch 1/5
80/80 [==============================] - 1s 6ms/step - loss: 0.9047 - accuracy: 0.7562
Epoch 2/5
80/80 [==============================] - 0s 6ms/step - loss: 0.2995 - accuracy: 0.9163
Epoch 3/5
80/80 [==============================] - 0s 6ms/step - loss: 0.2363 - accuracy: 0.9333
Epoch 4/5
80/80 [==============================] - 0s 6ms/step - loss: 0.2020 - accuracy: 0.9428
Epoch 5/5
40/40 [==============================] - 0s 3ms/step
Epoch 1/5
80/80 [==============================] - 1s 6ms/step - loss: 0.9567 - accuracy: 0.7140
E

80/80 [==============================] - 0s 5ms/step - loss: 0.2290 - accuracy: 0.9347
Epoch 4/10
80/80 [==============================] - 0s 5ms/step - loss: 0.1920 - accuracy: 0.9448
Epoch 5/10
80/80 [==============================] - 0s 5ms/step - loss: 0.1692 - accuracy: 0.9513
Epoch 6/10
80/80 [==============================] - 0s 5ms/step - loss: 0.1503 - accuracy: 0.9577
Epoch 7/10
80/80 [==============================] - 0s 6ms/step - loss: 0.1347 - accuracy: 0.9613
Epoch 8/10
80/80 [==============================] - 0s 5ms/step - loss: 0.1207 - accuracy: 0.9664
Epoch 9/10
80/80 [==============================] - 0s 6ms/step - loss: 0.1124 - accuracy: 0.9671
Epoch 10/10
40/40 [==============================] - 0s 3ms/step
Epoch 1/10
80/80 [==============================] - 1s 5ms/step - loss: 1.0235 - accuracy: 0.7131
Epoch 2/10
80/80 [==============================] - 0s 5ms/step - loss: 0.3218 - accuracy: 0.9101
Epoch 3/10
80/80 [==============================] - 0s 5ms/step 

40/40 [==============================] - 1s 8ms/step - loss: 1.3714 - accuracy: 0.6143
Epoch 2/5
40/40 [==============================] - 0s 8ms/step - loss: 0.4576 - accuracy: 0.8743
Epoch 3/5
40/40 [==============================] - 0s 7ms/step - loss: 0.3203 - accuracy: 0.9099
Epoch 4/5
40/40 [==============================] - 0s 8ms/step - loss: 0.2676 - accuracy: 0.9247
Epoch 5/5
20/20 [==============================] - 0s 5ms/step
Epoch 1/5
40/40 [==============================] - 1s 8ms/step - loss: 1.3039 - accuracy: 0.6458
Epoch 2/5
40/40 [==============================] - 0s 9ms/step - loss: 0.4407 - accuracy: 0.8796
Epoch 3/5
40/40 [==============================] - 0s 8ms/step - loss: 0.3218 - accuracy: 0.9096
Epoch 4/5
40/40 [==============================] - 0s 8ms/step - loss: 0.2748 - accuracy: 0.9232
Epoch 5/5
20/20 [==============================] - 0s 5ms/step
Epoch 1/5
40/40 [==============================] - 1s 8ms/step - loss: 1.4180 - accuracy: 0.5953
Epoch 2/5
4

40/40 [==============================] - 0s 9ms/step - loss: 0.2676 - accuracy: 0.9244
Epoch 5/10
40/40 [==============================] - 0s 9ms/step - loss: 0.2333 - accuracy: 0.9349
Epoch 6/10
40/40 [==============================] - 0s 9ms/step - loss: 0.2082 - accuracy: 0.9411
Epoch 7/10
40/40 [==============================] - 0s 9ms/step - loss: 0.1871 - accuracy: 0.9470
Epoch 8/10
40/40 [==============================] - 0s 9ms/step - loss: 0.1711 - accuracy: 0.9512
Epoch 9/10
40/40 [==============================] - 0s 9ms/step - loss: 0.1576 - accuracy: 0.9553
Epoch 10/10
20/20 [==============================] - 0s 5ms/step
Epoch 1/10
40/40 [==============================] - 2s 9ms/step - loss: 1.3925 - accuracy: 0.6313
Epoch 2/10
40/40 [==============================] - 0s 8ms/step - loss: 0.4683 - accuracy: 0.8727
Epoch 3/10
40/40 [==============================] - 0s 8ms/step - loss: 0.3189 - accuracy: 0.9104
Epoch 4/10
40/40 [==============================] - 0s 9ms/step 

In [9]:
print(grid_result.best_score_, grid_result.best_params_)

0.9580500000000001 {'batch_size': 500, 'epochs': 10, 'optimizer': 'Adam'}


In [10]:
df = pd.DataFrame(grid_result.cv_results_)

In [11]:
df.sort_values(['mean_test_score'],ascending=False)[['param_batch_size','param_epochs','param_optimizer','mean_test_score']].head(10)

,param_batch_size,param_epochs,param_optimizer,mean_test_score
5,500,10,Adam,0.958050
4,500,10,RMSprop,0.958033
3,500,10,SGD,0.956550
9,1000,10,SGD,0.950283
11,1000,10,Adam,0.949683
10,1000,10,RMSprop,0.949383
1,500,5,RMSprop,0.947533
2,500,5,Adam,0.944500
0,500,5,SGD,0.943750
6,1000,5,SGD,0.931700


### ДОМАШНЕЕ ЗАДАНИЕ

-Попробуйте обучить, нейронную сеть на Keras (рассмотренную на уроке) на датасете MNIST с другими параметрами. Напишите в комментарии к уроку:

-Какого результата вы добились от нейросети?

-Что помогло вам улучшить её точность?

-Поработайте с документацией Keras. Попробуйте найти полезные команды Keras, неразобранные на уроке

In [15]:
def my_model(optimizer='RMSprop'):
    model = Sequential([
      Dense(256, input_shape=(784,), activation='relu'),
      Dense(128, activation='relu'),
      Dense(10, activation='softmax')
    ])

    # Compile the model.
    model.compile(
      optimizer=optimizer,
      loss='categorical_crossentropy',
      metrics=['accuracy'],
    )
    return model

In [16]:
model = my_model()

# Train the model.
model.fit(train_images, to_categorical(train_labels), epochs=5, batch_size=32)

# Evaluate the model.
loss,metrics = model.evaluate(test_images, to_categorical(test_labels))
print('**************')
print(f'Loss - {np.round(loss,2)}, Metrics - {np.round(metrics,3)}')

Epoch 1/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2082 - accuracy: 0.9385
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0912 - accuracy: 0.9733
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0651 - accuracy: 0.9808
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0511 - accuracy: 0.9851
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0848 - accuracy: 0.9785
**************
Loss - 0.08, Metrics - 0.979


In [21]:
model_2 = my_model()

# Train the model.
model.fit(train_images, to_categorical(train_labels), epochs=10, batch_size=128, validation_split=0.3)

# Evaluate the model.
loss,metrics = model.evaluate(test_images, to_categorical(test_labels))
print('**************')
print(f'Loss - {np.round(loss,2)}, Metrics - {np.round(metrics,3)}')

Epoch 1/10
329/329 [==============================] - 2s 6ms/step - loss: 1.6816e-05 - accuracy: 1.0000 - val_loss: 0.0438 - val_accuracy: 0.9918
Epoch 2/10
329/329 [==============================] - 2s 6ms/step - loss: 1.3320e-05 - accuracy: 1.0000 - val_loss: 0.0437 - val_accuracy: 0.9917
Epoch 3/10
329/329 [==============================] - 2s 6ms/step - loss: 1.1733e-05 - accuracy: 1.0000 - val_loss: 0.0440 - val_accuracy: 0.9918
Epoch 4/10
329/329 [==============================] - 2s 6ms/step - loss: 1.0740e-05 - accuracy: 1.0000 - val_loss: 0.0442 - val_accuracy: 0.9917
Epoch 5/10
329/329 [==============================] - 2s 6ms/step - loss: 9.9948e-06 - accuracy: 1.0000 - val_loss: 0.0443 - val_accuracy: 0.9919
Epoch 6/10
329/329 [==============================] - 2s 6ms/step - loss: 9.2807e-06 - accuracy: 1.0000 - val_loss: 0.0446 - val_accuracy: 0.9918
Epoch 7/10
329/329 [==============================] - 2s 6ms/step - loss: 8.8044e-06 - accuracy: 1.0000 - val_loss: 0.0448 -

In [23]:
model_3 = my_model()

# Train the model.
model.fit(train_images, to_categorical(train_labels), epochs=5, batch_size=64, validation_split=0.2)

# Evaluate the model.
loss,metrics = model.evaluate(test_images, to_categorical(test_labels))
print('**************')
print(f'Loss - {np.round(loss,2)}, Metrics - {np.round(metrics,3)}')

Epoch 1/5
750/750 [==============================] - 4s 5ms/step - loss: 1.2712e-05 - accuracy: 1.0000 - val_loss: 0.0426 - val_accuracy: 0.9921
Epoch 2/5
750/750 [==============================] - 4s 5ms/step - loss: 8.7927e-06 - accuracy: 1.0000 - val_loss: 0.0429 - val_accuracy: 0.9922
Epoch 3/5
750/750 [==============================] - 4s 5ms/step - loss: 7.3723e-06 - accuracy: 1.0000 - val_loss: 0.0431 - val_accuracy: 0.9922
Epoch 4/5
750/750 [==============================] - 4s 5ms/step - loss: 6.5501e-06 - accuracy: 1.0000 - val_loss: 0.0431 - val_accuracy: 0.9921
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.1117 - accuracy: 0.9859
**************
Loss - 0.11, Metrics - 0.986


In [24]:
batch_size = [50, 200, 500, 1000, 1500]
epochs = [3, 5, 7, 12, 20]
optimizer = ['SGD', 'RMSprop', 'Adam']

In [25]:
model = KerasClassifier(my_model)
params= dict(batch_size=batch_size, epochs=epochs, optimizer=optimizer)
grid = GridSearchCV(model, params, refit=False, cv=3, scoring='accuracy')

grid_result = grid.fit(train_images, to_categorical(train_labels))
print(grid_result.best_score_, grid_result.best_params_)

Epoch 1/3
800/800 [==============================] - 4s 5ms/step - loss: 0.2645 - accuracy: 0.9222
Epoch 2/3
800/800 [==============================] - 4s 5ms/step - loss: 0.1103 - accuracy: 0.9659
Epoch 3/3
400/400 [==============================] - 1s 2ms/step
Epoch 1/3
800/800 [==============================] - 4s 5ms/step - loss: 0.2630 - accuracy: 0.9208
Epoch 2/3
800/800 [==============================] - 4s 5ms/step - loss: 0.1070 - accuracy: 0.9668
Epoch 3/3
400/400 [==============================] - 1s 2ms/step
Epoch 1/3
800/800 [==============================] - 5s 5ms/step - loss: 0.2639 - accuracy: 0.9219
Epoch 2/3
800/800 [==============================] - 4s 5ms/step - loss: 0.1044 - accuracy: 0.9675
Epoch 3/3
400/400 [==============================] - 1s 2ms/step
Epoch 1/3
800/800 [==============================] - 4s 4ms/step - loss: 0.2642 - accuracy: 0.9207
Epoch 2/3
800/800 [==============================] - 3s 4ms/step - loss: 0.1095 - accuracy: 0.9669
Epoch 3/3
400

800/800 [==============================] - 6s 5ms/step - loss: 0.2641 - accuracy: 0.9211
Epoch 2/7
800/800 [==============================] - 4s 5ms/step - loss: 0.1069 - accuracy: 0.9675
Epoch 3/7
800/800 [==============================] - 4s 5ms/step - loss: 0.0730 - accuracy: 0.9772
Epoch 4/7
800/800 [==============================] - 4s 5ms/step - loss: 0.0542 - accuracy: 0.9837
Epoch 5/7
800/800 [==============================] - 4s 5ms/step - loss: 0.0396 - accuracy: 0.9880
Epoch 6/7
800/800 [==============================] - 4s 5ms/step - loss: 0.0298 - accuracy: 0.9905
Epoch 7/7
400/400 [==============================] - 1s 3ms/step
Epoch 1/7
800/800 [==============================] - 5s 5ms/step - loss: 0.2590 - accuracy: 0.9230
Epoch 2/7
800/800 [==============================] - 4s 5ms/step - loss: 0.1067 - accuracy: 0.9678
Epoch 3/7
800/800 [==============================] - 4s 5ms/step - loss: 0.0723 - accuracy: 0.9775
Epoch 4/7
800/800 [==============================] - 4

800/800 [==============================] - 5s 6ms/step - loss: 0.0691 - accuracy: 0.9789
Epoch 4/12
800/800 [==============================] - 4s 5ms/step - loss: 0.0501 - accuracy: 0.9846
Epoch 5/12
800/800 [==============================] - 4s 5ms/step - loss: 0.0384 - accuracy: 0.9880
Epoch 6/12
800/800 [==============================] - 4s 6ms/step - loss: 0.0288 - accuracy: 0.9912
Epoch 7/12
800/800 [==============================] - 4s 6ms/step - loss: 0.0222 - accuracy: 0.9934
Epoch 8/12
800/800 [==============================] - 4s 6ms/step - loss: 0.0191 - accuracy: 0.9942
Epoch 9/12
800/800 [==============================] - 4s 5ms/step - loss: 0.0141 - accuracy: 0.9953
Epoch 10/12
800/800 [==============================] - 4s 6ms/step - loss: 0.0099 - accuracy: 0.9968
Epoch 11/12
800/800 [==============================] - 5s 6ms/step - loss: 0.0101 - accuracy: 0.9967
Epoch 12/12
400/400 [==============================] - 1s 3ms/step
Epoch 1/12
800/800 [======================

800/800 [==============================] - 5s 6ms/step - loss: 0.0152 - accuracy: 0.9954
Epoch 10/12
800/800 [==============================] - 5s 6ms/step - loss: 0.0116 - accuracy: 0.9963
Epoch 11/12
800/800 [==============================] - 5s 6ms/step - loss: 0.0101 - accuracy: 0.9969
Epoch 12/12
400/400 [==============================] - 2s 4ms/step
Epoch 1/12
800/800 [==============================] - 6s 6ms/step - loss: 0.2590 - accuracy: 0.9234
Epoch 2/12
800/800 [==============================] - 5s 6ms/step - loss: 0.1035 - accuracy: 0.9681
Epoch 3/12
800/800 [==============================] - 4s 5ms/step - loss: 0.0677 - accuracy: 0.9791
Epoch 4/12
800/800 [==============================] - 5s 6ms/step - loss: 0.0515 - accuracy: 0.9833
Epoch 5/12
800/800 [==============================] - 5s 6ms/step - loss: 0.0377 - accuracy: 0.9882
Epoch 6/12
800/800 [==============================] - 5s 6ms/step - loss: 0.0295 - accuracy: 0.9907
Epoch 7/12
800/800 [======================

800/800 [==============================] - 5s 7ms/step - loss: 0.0729 - accuracy: 0.9778
Epoch 4/20
800/800 [==============================] - 6s 7ms/step - loss: 0.0527 - accuracy: 0.9840
Epoch 5/20
800/800 [==============================] - 5s 7ms/step - loss: 0.0411 - accuracy: 0.9873
Epoch 6/20
800/800 [==============================] - 5s 7ms/step - loss: 0.0306 - accuracy: 0.9906
Epoch 7/20
800/800 [==============================] - 5s 6ms/step - loss: 0.0244 - accuracy: 0.9924
Epoch 8/20
800/800 [==============================] - 5s 6ms/step - loss: 0.0190 - accuracy: 0.9939
Epoch 9/20
800/800 [==============================] - 5s 6ms/step - loss: 0.0167 - accuracy: 0.9950
Epoch 10/20
800/800 [==============================] - 5s 6ms/step - loss: 0.0127 - accuracy: 0.9958
Epoch 11/20
800/800 [==============================] - 5s 6ms/step - loss: 0.0101 - accuracy: 0.9970
Epoch 12/20
800/800 [==============================] - 5s 6ms/step - loss: 0.0077 - accuracy: 0.9977
Epoch 13

...

27/27 [==============================] - 2s 30ms/step - loss: 0.8354 - accuracy: 0.7623
Epoch 2/7
27/27 [==============================] - 1s 31ms/step - loss: 0.3659 - accuracy: 0.8927
Epoch 3/7
27/27 [==============================] - 1s 31ms/step - loss: 0.2841 - accuracy: 0.9176
Epoch 4/7
27/27 [==============================] - 1s 30ms/step - loss: 0.2357 - accuracy: 0.9305
Epoch 5/7
27/27 [==============================] - 1s 31ms/step - loss: 0.1927 - accuracy: 0.9434
Epoch 6/7
27/27 [==============================] - 1s 29ms/step - loss: 0.1698 - accuracy: 0.9496
Epoch 7/7
14/14 [==============================] - 0s 14ms/step
Epoch 1/7
27/27 [==============================] - 2s 31ms/step - loss: 0.8835 - accuracy: 0.7507
Epoch 2/7
27/27 [==============================] - 1s 30ms/step - loss: 0.3662 - accuracy: 0.8949
Epoch 3/7
27/27 [==============================] - 1s 30ms/step - loss: 0.2796 - accuracy: 0.9181
Epoch 4/7
27/27 [==============================] - 1s 30ms/step 

27/27 [==============================] - 1s 30ms/step - loss: 0.1631 - accuracy: 0.9531
Epoch 7/12
27/27 [==============================] - 1s 31ms/step - loss: 0.1436 - accuracy: 0.9574
Epoch 8/12
27/27 [==============================] - 1s 31ms/step - loss: 0.1228 - accuracy: 0.9637
Epoch 9/12
27/27 [==============================] - 1s 30ms/step - loss: 0.1100 - accuracy: 0.9677
Epoch 10/12
27/27 [==============================] - 1s 30ms/step - loss: 0.1004 - accuracy: 0.9706
Epoch 11/12
27/27 [==============================] - 1s 30ms/step - loss: 0.0892 - accuracy: 0.9735
Epoch 12/12
14/14 [==============================] - 0s 16ms/step
Epoch 1/12
27/27 [==============================] - 2s 32ms/step - loss: 0.8748 - accuracy: 0.7535
Epoch 2/12
27/27 [==============================] - 1s 33ms/step - loss: 0.3636 - accuracy: 0.8942
Epoch 3/12
27/27 [==============================] - 1s 32ms/step - loss: 0.2824 - accuracy: 0.9180
Epoch 4/12
27/27 [==============================] - 

14/14 [==============================] - 0s 14ms/step
Epoch 1/20
27/27 [==============================] - 2s 31ms/step - loss: 0.8629 - accuracy: 0.7561
Epoch 2/20
27/27 [==============================] - 1s 30ms/step - loss: 0.3761 - accuracy: 0.8911
Epoch 3/20
27/27 [==============================] - 1s 31ms/step - loss: 0.2944 - accuracy: 0.9139
Epoch 4/20
27/27 [==============================] - 1s 30ms/step - loss: 0.2445 - accuracy: 0.9288
Epoch 5/20
27/27 [==============================] - 1s 30ms/step - loss: 0.2068 - accuracy: 0.9391
Epoch 6/20
27/27 [==============================] - 1s 30ms/step - loss: 0.1772 - accuracy: 0.9484
Epoch 7/20
27/27 [==============================] - 1s 30ms/step - loss: 0.1515 - accuracy: 0.9560
Epoch 8/20
27/27 [==============================] - 1s 30ms/step - loss: 0.1418 - accuracy: 0.9577
Epoch 9/20
27/27 [==============================] - 1s 31ms/step - loss: 0.1254 - accuracy: 0.9635
Epoch 10/20
27/27 [==============================] - 1s

14/14 [==============================] - 0s 19ms/step
Epoch 1/20
27/27 [==============================] - 2s 30ms/step - loss: 0.8575 - accuracy: 0.7612
Epoch 2/20
27/27 [==============================] - 1s 31ms/step - loss: 0.3695 - accuracy: 0.8932
Epoch 3/20
27/27 [==============================] - 1s 30ms/step - loss: 0.2761 - accuracy: 0.9213
Epoch 4/20
27/27 [==============================] - 1s 31ms/step - loss: 0.2353 - accuracy: 0.9313
Epoch 5/20
27/27 [==============================] - 1s 29ms/step - loss: 0.2008 - accuracy: 0.9411
Epoch 6/20
27/27 [==============================] - 1s 31ms/step - loss: 0.1745 - accuracy: 0.9484
Epoch 7/20
27/27 [==============================] - 1s 31ms/step - loss: 0.1531 - accuracy: 0.9550
Epoch 8/20
27/27 [==============================] - 1s 30ms/step - loss: 0.1357 - accuracy: 0.9610
Epoch 9/20
27/27 [==============================] - 1s 31ms/step - loss: 0.1236 - accuracy: 0.9638
Epoch 10/20
27/27 [==============================] - 1s

14/14 [==============================] - 0s 16ms/step
Epoch 1/20
27/27 [==============================] - 2s 30ms/step - loss: 0.8615 - accuracy: 0.7572
Epoch 2/20
27/27 [==============================] - 1s 30ms/step - loss: 0.3812 - accuracy: 0.8889
Epoch 3/20
27/27 [==============================] - 1s 30ms/step - loss: 0.2828 - accuracy: 0.9186
Epoch 4/20
27/27 [==============================] - 1s 31ms/step - loss: 0.2367 - accuracy: 0.9310
Epoch 5/20
27/27 [==============================] - 1s 30ms/step - loss: 0.2027 - accuracy: 0.9403
Epoch 6/20
27/27 [==============================] - 1s 31ms/step - loss: 0.1731 - accuracy: 0.9497
Epoch 7/20
27/27 [==============================] - 1s 31ms/step - loss: 0.1512 - accuracy: 0.9567
Epoch 8/20
27/27 [==============================] - 1s 31ms/step - loss: 0.1301 - accuracy: 0.9625
Epoch 9/20
27/27 [==============================] - 1s 31ms/step - loss: 0.1186 - accuracy: 0.9649
Epoch 10/20
27/27 [==============================] - 1s

In [26]:
print(grid_result.best_score_, grid_result.best_params_)

0.9785833333333332 {'batch_size': 200, 'epochs': 20, 'optimizer': 'Adam'}


In [27]:
df = pd.DataFrame(grid_result.cv_results_)

In [28]:
df.sort_values(['mean_test_score'],ascending=False)[['param_batch_size','param_epochs','param_optimizer','mean_test_score']].head(10)

,param_batch_size,param_epochs,param_optimizer,mean_test_score
29,200,20,Adam,0.978583
14,50,20,Adam,0.978567
28,200,20,RMSprop,0.978117
13,50,20,RMSprop,0.978117
27,200,20,SGD,0.977717
26,200,12,Adam,0.977033
12,50,20,SGD,0.976933
44,500,20,Adam,0.976567
42,500,20,SGD,0.976283
43,500,20,RMSprop,0.976150


### Вывод:

Лучший результат показала модель с параметрами({'batch_size': 200, 'epochs': 20, 'optimizer': 'Adam'})
Так же можно сделать вывод, что валидационная выборка улучшает результат обучения модели, снижая возможность переобучения